In [1]:
# Create a new set of 2,000 random latitudes and longitudes.

In [3]:
# Import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
from config import weather_api_key
import requests
import time
from scipy.stats import linregress
import gmaps
from config import g_key


In [4]:
# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

Beginning Data Retrieval     
-----------------------------


In [5]:
# Create 2000 random latitude and longitude coordinates 
latitudes = np.random.uniform(-90, 90, size = 2000)
longitudes = np.random.uniform(-180, 180, size = 2000)
coordinates = zip(latitudes, longitudes)
coordinates

# Add coordinates to a list 
coordinates_lst = list(coordinates)

In [6]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for coordinate in coordinates_lst:
    city = citipy.nearest_city(coordinate[0],coordinate[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)


755

In [7]:
# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Make an API request for each city 
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_city = city.title()
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']

        # Fill City Data list 
        city_data.append({"City": city_city,
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Description": city_desc,
                         })
        
    #If an error occurs
    except: 
        print("City not found. Skipping")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Processing Record 1 of Set 1 | tautira
Processing Record 2 of Set 1 | chomun
City not found. Skipping
Processing Record 3 of Set 1 | chokurdakh
Processing Record 4 of Set 1 | puerto ayora
Processing Record 5 of Set 1 | kapaa
Processing Record 6 of Set 1 | sobolevo
Processing Record 7 of Set 1 | taolanaro
City not found. Skipping
Processing Record 8 of Set 1 | butaritari
Processing Record 9 of Set 1 | hobart
Processing Record 10 of Set 1 | cartagena del chaira
Processing Record 11 of Set 1 | upernavik
Processing Record 12 of Set 1 | narsaq
Processing Record 13 of Set 1 | new norfolk
Processing Record 14 of Set 1 | punta arenas
Processing Record 15 of Set 1 | saint-joseph
Processing Record 16 of Set 1 | mayo
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | port lincoln
Processing Record 19 of Set 1 | zhanakorgan
Processing Record 20 of Set 1 | sioux lookout
Processing Record 21 of Set 1 | qaanaaq
Processing Record 22 of Set 1 | verkh-usugli
Processing Record 23 of 

In [8]:
len(city_data)

697

In [9]:
# Add the data to a list in a dict then at the DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Tautira,PF,2022-09-21 20:27:05,-17.7333,-149.1500,77.32,75,18,16.46,few clouds
1,Chokurdakh,RU,2022-09-21 20:25:54,70.6333,147.9167,32.47,93,100,9.78,overcast clouds
2,Puerto Ayora,EC,2022-09-21 20:27:06,-0.7393,-90.3518,76.96,89,76,11.99,broken clouds
3,Kapaa,US,2022-09-21 20:27:07,22.0752,-159.3190,84.52,71,0,11.50,clear sky
4,Sobolevo,RU,2022-09-21 20:27:07,54.4299,31.8996,46.67,96,100,8.99,overcast clouds
5,Butaritari,KI,2022-09-21 20:27:08,3.0707,172.7902,79.93,75,93,4.54,light rain
6,Hobart,AU,2022-09-21 20:27:08,-42.8794,147.3294,52.05,91,33,3.44,scattered clouds
7,Cartagena Del Chaira,CO,2022-09-21 20:22:59,1.3349,-74.8429,86.59,59,100,0.87,overcast clouds
8,Upernavik,GL,2022-09-21 20:23:16,72.7868,-56.1549,33.73,72,100,5.26,overcast clouds
9,Narsaq,GL,2022-09-21 20:27:09,60.9167,-46.0500,48.42,61,30,3.47,scattered clouds


In [10]:
# Create CSV file of DF
csv_weather_file = "weatherpy_database.csv"
# Export CSV file 
city_data_df.to_csv(csv_weather_file, index_label="City_ID")

In [ ]:
###### everything below is practice/incorrect ######

In [11]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
#city_data_df.head()

In [12]:
# Create a new set of 2,000 random latitudes and longitudes
# Create a set of random latitude and longitude combinations.
#lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
#lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Unpack lat_lngs zip object into a list. This way, we only need to create a set of random latitudes and longitudes once. 
#lat_lngs = zip(lats, lngs)
#lat_lngs

# One can only unzip a zipped tuple once before it is removed from the computer's memory. 
# Make sure to unzip the latitudes and longitudes into the coordinates list before moving on.


In [4]:

# Starting URL for Weather Map API Call.
# url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)

In [14]:
# Add the latitudes and longitudes to a list.
#coordinates = list(lat_lngs)
#print (coordinates)

In [15]:
# Get the nearest city using the citipy module.

# Create a list for holding the cities.
#cities = []
# Identify the nearest city for each latitude and longitude combination.
#for coordinate in coordinates:
#    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
#    if city not in cities:
#        cities.append(city)
# Print the city count to confirm sufficient count.
#len(cities)

In [7]:
# Perform an API call with the OpenWeatherMap.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [16]:
# Retrieve the following information from the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
#record_count = 1
#set_count = 1

# Loop through all the cities in the list.
#for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
#    if (i % 50 == 0 and i >= 50):
#        set_count += 1
#        record_count = 1

        
    # Create endpoint URL with each city.
#    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
#    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
#    record_count += 1
    
    
    
      # Run an API request for each of the cities.
#    try:
#        # Parse the JSON and retrieve data.
#        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
#        city_lat = city_weather["coord"]["lat"]
#        city_lng = city_weather["coord"]["lon"]
#        city_country = city_weather["sys"]["country"]
#        city_max_temp = city_weather["main"]["temp_max"]
#        city_humidity = city_weather["main"]["humidity"]
#        city_clouds = city_weather["clouds"]["all"]
#        city_wind = city_weather["wind"]["speed"]
#        current_description = city_weather['weather']['description']
        
        
        # Convert the date to ISO
#        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
#        city_data.append({"City": city.title(),
#                          "Country": city_country,
#                          "Lat": city_lat,
#                          "Lng": city_lng,
#                          "Max Temp": city_max_temp,
#                          "Humidity": city_humidity,
#                          "Cloudiness": city_clouds,
#                          "Wind Speed": city_wind })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mataura
City not found. Skipping...
Processing Record 2 of Set 1 | ossora
City not found. Skipping...
Processing Record 3 of Set 1 | bambous virieux
City not found. Skipping...
Processing Record 4 of Set 1 | carnarvon
City not found. Skipping...
Processing Record 5 of Set 1 | norman wells
City not found. Skipping...
Processing Record 6 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 7 of Set 1 | busselton
City not found. Skipping...
Processing Record 8 of Set 1 | mahebourg
City not found. Skipping...
Processing Record 9 of Set 1 | tasiilaq
City not found. Skipping...
Processing Record 10 of Set 1 | rikitea
City not found. Skipping...
Processing Record 11 of Set 1 | chokurdakh
City not found. Skipping...
Processing Record 12 of Set 1 | rabo de peixe
City not found. Skipping...
Processing Record 13 of Set 1 | la ronge
City not found. Skipping...
Processing Record 14 of Set 1 |

In [17]:
# Tally the number of cities in the city_data array of dictionaries
len(city_data)


0

In [10]:
# 6.2.7


# Convert the array of dictionaries to a Pandas DataFrame.
# Add the data to a new DataFrame.


city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Mataura,NZ,-46.1927,168.8643,42.66,95,27,1.52
1,Ossora,RU,59.2353,163.0719,43.75,69,33,28.57
2,Bambous Virieux,MU,-20.3428,57.7575,72.21,83,75,13.80
3,Carnarvon,AU,-24.8667,113.6333,63.66,77,0,20.40
4,Norman Wells,CA,65.2820,-126.8329,48.24,87,75,8.05
5,Busselton,AU,-33.6500,115.3333,46.29,68,33,7.47
6,Mahebourg,MU,-20.4081,57.7000,72.25,83,75,13.80
7,Tasiilaq,GL,65.6145,-37.6368,37.56,94,63,4.94
8,Rikitea,PF,-23.1203,-134.9692,72.88,75,2,14.50
9,Chokurdakh,RU,70.6333,147.9167,32.47,93,100,9.78


In [11]:
import csv

In [12]:
# Export the DataFrame as WeatherPy_Database.csv into the Weather_Database folder (5 pt)

# create output file
output_data_file = "city_data.csv"

# Export city_data to csv
city_data_df.to_csv(output_data_file, index_label = "WeatherPy_Database")

In [15]:
# Add data to a new DataFrame
cities_weather_df = pd.DataFrame(city_data)
# Reorder the columns
cities_weather_df=cities_weather_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description" ]]
cities_weather_df.head()

KeyError: "['Current Description'] not in index"

In [14]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the dataframe to csv
cities_weather_df.to_csv(output_data_file, index_label="City_ID")